In [31]:
import pandas as pd
import numpy as np
import scipy as sc

from datetime import datetime, timedelta
from statsmodels.regression.linear_model import OLS

import warnings 
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [63]:
# import firm
d1 = pd.read_excel('11beta.xlsx')
d1.dropna(inplace=True)
d1.sort_values(['GVKey','DataDate'], inplace=True)


# 31 to 1
d1['DataDate']=d1['DataDate']-pd.offsets.MonthBegin(1)

# %
d1['x1'] = d1.groupby('GVKey')['MkVal'].pct_change()
d1.dropna(inplace=True)

# clean
d5 = d1.loc[:, ['GVKey','DataDate','x1']]
d5 = d5.rename(columns={'DataDate': 'Data'})
d5 = d5[d5['x1']<0.5]
d5 = d5[d5['x1']>-0.5]

# dohodnosti rinka 
y= pd.read_csv('yahoo.csv',sep=',')
y['Close1'] = y['Close'].shift(+1)
y['x']=(y['Close']-y['Close1'])/y['Close1']
y.dropna(inplace=True)
y1 = y[['Date','x']]
y1 = y1.rename(columns={'Date': 'Data'})
y1['Data']= pd.DatetimeIndex(y1['Data'])

# merge market + firm
dd = d5.merge(y1)

# reg 10
z10 = ('1990-1-1','1991-1-1','1992-1-1','1993-1-1','1994-1-1',
       '1995-1-1','1996-1-1','1997-1-1','1998-1-1',
       '1999-1-1','2000-1-1','2001-1-1','2002-1-1',
       '2003-1-1','2004-1-1','2005-1-1','2006-1-1',
       '2007-1-1','2008-1-1','2009-1-1','2010-1-1',
       '2011-1-1','2012-1-1','2013-1-1','2014-1-1',
       '2015-1-1','2016-1-1','2017-1-1','2018-1-1')

# reg 5
z5 = ('1995-1-1','1996-1-1','1997-1-1','1998-1-1','1999-1-1',
      '2000-1-1','2001-1-1','2002-1-1','2003-1-1',
      '2004-1-1','2005-1-1','2006-1-1','2007-1-1',
      '2008-1-1','2009-1-1','2010-1-1','2011-1-1',
      '2012-1-1','2013-1-1','2014-1-1','2015-1-1',
      '2016-1-1','2017-1-1','2018-1-1')

res10 = pd.DataFrame({'GVKey': [1],
                    'Beta': [1],
                    'Year':[1]},
                    index=[0])

# reg 10

for i in range(19):
    g = (dd[(dd['Data'] >= z10[i]) & (dd['Data'] <= z10[i+10])]
             .groupby(['GVKey'])[['x1','x']]
             .apply(lambda x: OLS.from_formula('x1 ~ x', x)
                                 .fit()
                                 .params['x'])
             .reset_index(name='Beta'))
    g['Year'] = 2000 + i
    res10 = res10.append(g)
res10.rename(columns= {'GVKey': 'FIRM'}, inplace=True)  

# reg 5 int 5
res5 = pd.DataFrame({'GVKey': [1],
                    'Beta': [1],
                    'Year':[1]},
                    index=[0])
for i in range(19):
    g = (dd[(dd['Data'] >= z5[i]) & (dd['Data'] <= z5[i+5])]
             .groupby(['GVKey'])[['x1','x']]
             .apply(lambda x: OLS.from_formula('x1 ~ x', x)
                                 .fit()
                                 .params['x'])
             .reset_index(name='Beta'))
    g['Year'] = 2000 + i
    res5 = res5.append(g)  
res5.rename(columns= {'GVKey': 'FIRM'}, inplace=True)

res10 = res10[['FIRM', 'Year', 'Beta']]
res10 = res10.rename(columns = {'Beta': 'Beta10'})
res5 = res5[['FIRM', 'Year', 'Beta']]
res5 = res5.rename(columns = {'Beta': 'Beta5'})

# itog beta
merge_beta = res5.merge(res10)
merge_beta = merge_beta.sort_values(['Year'])
merge_beta = merge_beta.drop(axis=0, index = 0)

merge_beta['beta'] = (merge_beta['Beta5'] + merge_beta['Beta10'])/2

In [64]:
naming = pd.read_excel('my_excel_file.xlsx')

list = []
for element in merge_beta['FIRM']:
    if element in np.array(naming['GVKey']):
        list.append(1)
    else:
        list.append(0)

merge_beta['indexing'] = list
merge_beta = merge_beta[merge_beta.indexing == 1]

merge_beta['company'] = 'new_name'
merge_beta['label'] = 'new_label'
for element in np.array(naming['GVKey']):
    merge_beta['company'][merge_beta['FIRM'] == element] = np.array(naming['Company'][naming['GVKey'] == element])[0]
    merge_beta['label'][merge_beta['FIRM'] == element] = np.array(naming['Label'][naming['GVKey'] == element])[0]
merge_beta = merge_beta[['Year', 'company', 'label', 'Beta5', 'Beta10', 'beta']]
merge_beta.dropna(inplace= True)

outliers = np.array(merge_beta[merge_beta.groupby('company')['beta'].apply(lambda x: \
                    np.abs(x - x.median()) > (2.2 * 1.4826 * x.mad()))].any(axis=1).index)
merge_beta.drop(outliers, inplace = True)
merge_beta.to_excel('beta.xlsx')

In [125]:
merge_beta = pd.read_excel('beta.xlsx')

# Подтягиваем табличку с премией рынка и безрисковыми доходностями
damodaran = pd.read_excel('http://www.stern.nyu.edu/~adamodar/pc/datasets/histimpl.xls', 
                          sheet_name = 'Historical Impl Premiums', header = 6)
year = np.arange(1961, 2019, 1)
list = np.arange(1,59,1)
rf = damodaran.loc[list, ['T.Bond Rate']]
ERP = damodaran.loc[list, ['Implied Premium (FCFE)']]
help = pd.DataFrame(np.column_stack([rf, ERP, year]), 
                               columns=['Rf', 'ERP', 'Year'])
help_1 = help[['Year','ERP', 'Rf' ]]
Re = merge_beta.merge(help_1)

Re['Re'] = Re['Rf'] + Re['ERP'] * Re['beta']
Re = Re.sort_values(['company', 'Year'])
Re.reset_index(inplace=True)
Re.drop('index', axis=1, inplace=True)
Re = Re.assign(Year=lambda d: d.pop('Year').astype('int'))

In [159]:
Re.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7658 entries, 0 to 7657
Data columns (total 7 columns):
company    7658 non-null object
label      7658 non-null object
beta       7658 non-null float64
ERP        7658 non-null float64
Rf         7658 non-null float64
Re         7658 non-null float64
Year       7658 non-null int32
dtypes: float64(4), int32(1), object(2)
memory usage: 389.0+ KB


In [140]:
xu = pd.read_excel('RRR.xlsx')
xu.dropna(inplace=True)
xu = xu.rename(columns = {'ICR': 'RATING'})
xu.dropna(inplace=True)


bins = [-1000000, 0.1999,0.6499,0.7999,1.2499,1.4999,1.7499,1.9999,2.2499,2.4999,2.9999,4.2499,5.4999,6.4999,8.4999,10000000]
labels = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15']
xu['FORMULA'] = pd.cut(xu['RATING'], bins=bins, labels=labels, right=True)
formulas = """
F15=0.006
F14=0.008
F13=0.01
F12=0.011
F11=0.0125
F10=0.016
F9=0.025
F8=0.03
F7=0.0375
F6=0.045
F5=0.055
F4=0.065
F3=0.08
F2=0.105
F1=0.14
  
"""
x = xu.eval(formulas).dropna(subset=['FORMULA'])
x['RES'] = x.lookup(x.index, x['FORMULA'])
xu.loc[xu['RATING'].notna(), 'RES'] = x['RES']
spread = xu[['DataDate','GVKey','RES']]
spread = spread.rename(columns = {'RES': 'spread','DataDate': "Year",'GVKey':'FIRM' })
spread['Year'] = pd.DatetimeIndex(spread['Year']).year

naming = pd.read_excel('my_excel_file.xlsx')

list = []
for element in spread['FIRM']:
    if element in np.array(naming['GVKey']):
        list.append(1)
    else:
        list.append(0)

spread['indexing'] = list
spread = spread[spread.indexing == 1]

spread['company'] = 'new_name'
spread['label'] = 'new_label'
for element in np.array(naming['GVKey']):
    spread['company'][spread['FIRM'] == element] = np.array(naming['Company'][naming['GVKey'] == element])[0]
    spread['label'][spread['FIRM'] == element] = np.array(naming['Label'][naming['GVKey'] == element])[0]
spread = spread[['Year', 'company', 'label', 'spread']]

In [179]:
WACC = pd.merge(Re, spread, how = 'left')
WACC = WACC.drop_duplicates()
WACC['spread'] = WACC.groupby('company')['spread'].fillna(method='ffill')
WACC['spread'] = WACC.groupby('company')['spread'].fillna(method='bfill')
WACC['Rd'] = WACC['Rf'] + WACC['spread']
WACC = WACC.set_index(keys='Year')
WACC = WACC.reset_index()
WACC.dropna(inplace =True)

In [182]:
WACC.to_excel('wacc.xlsx')